### Practice: Parameter Efficient Fine-Tuning
In this notebook, you're gonna fine-tune large language models within limited GPU memory.

In [1]:
#%pip install --quiet torch==2.1.0 transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2

import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/daozerova/.conda/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import transformers
import torch
transformers.__version__, torch.__version__

('4.46.3', '2.1.2')

In [23]:
model_name = 'Enoch/llama-7b-hf'

# loading Llama tokenizer ...
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

# ... and the model itself
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable()  # only store a small subset of activations, re-compute the rest.
model.enable_input_require_grads()     # override an implementation quirk in gradient checkpoints that disables backprop unless inputs require grad
# more on gradient checkpointing: https://pytorch.org/docs/stable/checkpoint.html https://arxiv.org/abs/1604.06174

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 33/33 [00:16<00:00,  1.95it/s]
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


### Prompt tuning: the story of a fox (2 pts)

![img](https://i.imgur.com/Ux3qQAu.png) (source: theodd1souts.fandom.com)

In [4]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print('BATCH', batch)

for i in range(10):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))

BATCH {'input_ids': tensor([[    1,   319,  4996, 17354,  1701, 29916]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}

Output: <s>A quick brown fox jumps over the lazy dog.
A quick


What a blatant lie! This particular fox assures you that it didn't in fact jump over the lazy dog. No, sir! The fox was just minding its own business. __Your task is to train the model to say truth: no dog was jumped over today.__

In [5]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)
print(batch, batch.input_ids.shape)
outputs = model(**batch)

next_word_logits = outputs.logits[:, :-1]
#print('outputs', outputs)
print('next_word_logits', next_word_logits.shape)
true_next_tokens = batch['input_ids'][:, 1:]
loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))

print("Loss:", loss)

{'input_ids': tensor([[    1,   319,  4996, 17354,  1701, 29916,  1258,   451, 12500,   975,
           278, 17366, 11203, 29889, 19065, 29892,   393, 11203,   553,  9841,
           372,  8763, 29991]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')} torch.Size([1, 23])
next_word_logits torch.Size([1, 22, 32000])
Loss: tensor(3.0725, device='cuda:0', grad_fn=<NllLossBackward0>)


Except, we can't train the entire model - that would be 28GB gradients in float32. Instead, let's run [prompt tuning](https://arxiv.org/abs/2104.08691).

![img](https://i.imgur.com/VwNNKnb.png)


In [6]:
class WordEmbeddingsWithLearnedPrompts(nn.Module):
    """
    To perform prompt tuning, you will need to replace model's original word embeddings with a layer - THIS layer
     - that inserts trainable prompts instead of the first N token embeddings. """

    def __init__(self, word_embeddings: nn.Embedding, num_prompts: int):
        super().__init__()
        self.original_word_embeddings = word_embeddings
        self.num_prompts = num_prompts
        self.learnable_prompts = nn.Parameter(
            torch.randn(1, num_prompts, word_embeddings.embedding_dim), requires_grad=True)

    def forward(self, input_ids: torch.LongTensor):
        # input_ids shape: [batch_size, seq length]
        assert input_ids.dtype == torch.int64
        assert input_ids.shape[1] > self.num_prompts
        assert torch.all(input_ids[:, :self.num_prompts] == tokenizer.pad_token_id).item(), "don't forget to prepend several BOS tokens to input_ids"

        # Your task: embed input_ids, but replace the first :num_prompts: tokens with self.learnable_prompts
        # This is because we will prepend :num_prompts: padding tokens at the beginning

        # After you are done, you must produce a word embedding vector for each token in input_ids,
        # except that the first :num_prompts: vectors should equal learnable_prompts;
        # any additional vectors after first :num_prompts: ones should be embedded as usual
        # Note: since you're dealing with trainable params, please torch.cat instead of item assignment

        #<YOUR CODE HERE>
        #print(input_ids, input_ids.shape)
        embedded_prompt = self.original_word_embeddings(input_ids)
        #print(embedded_prompt, embedded_prompt.shape, embedded_prompt[:, self.num_prompts:].shape)
        replaced_prompt = torch.cat([self.learnable_prompts, embedded_prompt[:, self.num_prompts:]], dim = 1)
        #print(replaced_prompt.shape)
        return replaced_prompt #your_outputs_with_prompts_as_per_instructions_above

In [7]:
num_prompts = 16
test_emb_layer = WordEmbeddingsWithLearnedPrompts(model.model.embed_tokens, num_prompts=num_prompts).to(device)
test_input_ids = tokenizer("a cat say on a may", return_tensors='pt')['input_ids'].to(device)

space_for_prompts = torch.full([len(test_input_ids), num_prompts], fill_value=tokenizer.pad_token_id,
                               dtype=torch.int64, device=device)
test_inputs_with_prompts = torch.cat([space_for_prompts, test_input_ids], dim=1)

with torch.cuda.amp.autocast():
    test_prompt_embeddings = test_emb_layer(test_inputs_with_prompts)

assert test_prompt_embeddings.shape[:2] == test_inputs_with_prompts.shape
assert test_prompt_embeddings.shape[-1] == model.config.hidden_size
assert torch.allclose(test_prompt_embeddings[:, :num_prompts], test_emb_layer.learnable_prompts.float())
assert torch.allclose(test_prompt_embeddings[:, num_prompts:], model.model.embed_tokens(test_input_ids).float())
print("Looks legit!")

Looks legit!


__Now that it works,__ let's inject learnable prompts into the main model and teach it about foxes.

In [8]:
assert isinstance(model.model.embed_tokens, nn.Embedding), "you have already replaced the embedding layer. If the replacement is broken, please reload the model"
save_emb_tokens_matr = model.model.embed_tokens
model.model.embed_tokens = WordEmbeddingsWithLearnedPrompts(model.model.embed_tokens, num_prompts=num_prompts).to(device)

opt = torch.optim.Adam([model.model.embed_tokens.learnable_prompts], lr=0.01)

In [9]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)
space_for_prompts = torch.full([len(test_input_ids), num_prompts], fill_value=tokenizer.pad_token_id,
                               dtype=torch.int64, device=device)
batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)

while loss.item() > 0.1:
    opt.zero_grad()
    outputs = model(**batch)
    next_word_logits = outputs.logits[:, num_prompts : -1, :]
    true_next_tokens = batch['input_ids'][:, num_prompts + 1:]
    loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
    print("Loss:", loss)
    loss.backward()
    opt.step()

assert loss.item() <= 0.1
print("Good job!")

Loss: tensor(7.2487, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(6.5535, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(6.1169, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(5.2927, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(4.4734, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.5732, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.2477, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.0043, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.8375, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.7687, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.6666, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.6433, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.5917, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.5186, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.4707, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tens

In [10]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)


for i in range(15):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0, num_prompts:].cpu().numpy().tolist()))

# if you did everything right, the model will deny that the fox jumped over the lazy dog


Output: <s>A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it


### Попробуем проинтерпретировать обученные параметры промптов: ###

In [11]:
learned_prompts = torch.clone(model.model.embed_tokens.learnable_prompts[0]).cpu()
emb_lookup = torch.clone(model.model.embed_tokens.original_word_embeddings.weight).cpu()
emb_lookup.shape, learned_prompts.shape

(torch.Size([32000, 4096]), torch.Size([16, 4096]))

In [13]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
similar_vecs, cos_similarity_save = [], []
for i in range(learned_prompts.shape[0]):
    cos_sim = cos(learned_prompts[i].unsqueeze(0).repeat(emb_lookup.shape[0], 1), emb_lookup)
    max_idx = cos_sim.argmax()
    cos_similarity_save.append(cos_sim[max_idx])
    similar_vecs.append(max_idx.item())

similar_vecs

[23799,
 14566,
 11699,
 17018,
 22366,
 12471,
 58,
 27158,
 28635,
 4369,
 1,
 3022,
 27878,
 1008,
 26273,
 18712]

### Нашли в embedding lookup table наиболее схожие вектора, получился промпт: ###

In [15]:
learned_prompt = tokenizer.decode(similar_vecs)
learned_prompt

' GT ihren Has rif imply environ7 Fernández bridolf<s>conc recomater Muséeburn'

### Проверим: подадим в модель текст про лису с обченным промптом ###

In [19]:
batch = tokenizer(learned_prompt + '<s>' + prompt, return_tensors='pt', return_token_type_ids=False, add_special_tokens=False).to(device)
for i in range(15):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))


Output:  GT ihren Has rif imply environ7 Fernández bridolf<s> conc recomater Muséeburn<s>A quick brown fox jumps over the lazy dog.
The quick brown fox jumps


In [20]:
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
batch['input_ids'] = torch.cat([torch.tensor(similar_vecs, device=batch['input_ids'].device).unsqueeze(0), batch['input_ids']], dim=1)
batch['attention_mask'] = torch.cat([torch.ones_like(torch.tensor(similar_vecs, device=batch['input_ids'].device).unsqueeze(0)), batch['attention_mask']], dim=1)

for i in range(15):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))


Output:  GT ihren Has rif imply environ7 Fernández bridolf<s> conc recomater Muséeburn<s>A quick brown fox jumps over the lazy dog.
The fox is a member of


### В результате обученный промпт не дал желаемых результатов ###

In [14]:
cos_similarity_save

[tensor(0.0678, grad_fn=<SelectBackward0>),
 tensor(0.0684, grad_fn=<SelectBackward0>),
 tensor(0.0776, grad_fn=<SelectBackward0>),
 tensor(0.0722, grad_fn=<SelectBackward0>),
 tensor(0.0625, grad_fn=<SelectBackward0>),
 tensor(0.0687, grad_fn=<SelectBackward0>),
 tensor(0.0709, grad_fn=<SelectBackward0>),
 tensor(0.0638, grad_fn=<SelectBackward0>),
 tensor(0.0620, grad_fn=<SelectBackward0>),
 tensor(0.0605, grad_fn=<SelectBackward0>),
 tensor(0.0616, grad_fn=<SelectBackward0>),
 tensor(0.0634, grad_fn=<SelectBackward0>),
 tensor(0.0690, grad_fn=<SelectBackward0>),
 tensor(0.0677, grad_fn=<SelectBackward0>),
 tensor(0.0573, grad_fn=<SelectBackward0>),
 tensor(0.0633, grad_fn=<SelectBackward0>)]

Если посмотреть на cosine similarity, то можно увидеть, что в словре модели просто не нашлось достаточно похожих токенов, поэтому качество плохое

### Using HuggingFace PEFT (2 points)

[`peft`](https://huggingface.co/docs/peft/index) is a transformer's sister library that allows you to apply various __p__arameter __e__fficient __f__ine-__t__uning methods to pre-trained transformers. The library imlements both prompt tuning, prefix tuning, as well as several adapter-based techniques under a common interface:



In [17]:
import peft
assert isinstance(model.model.embed_tokens, nn.Embedding), "please reload the model"

peft_config = peft.PromptTuningConfig(task_type=peft.TaskType.CAUSAL_LM, num_virtual_tokens=16)
model = peft.get_peft_model(model, peft_config)  # note: for most peft methods, this line also modifies model in-place
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print("Total parameters (excluding quantization):", sum(p.numel() for p in model.parameters()))

Trainable parameters: 65536
Total parameters (excluding quantization): 3500478464


In [ ]:
# Your task: optimize the PEFT-wrapped model to achieve next token prediction loss < 0.1, but this time using PEFT
# Please note: you no longer need to prepend PAD tokens, but you still need to skip :num_virtual_tokens: first logits.
# Finally, generate the sentence to make sure that the model learned the truth.

In [30]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)
opt = torch.optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.01)


num_virtual_tokens = 16
while loss.item() > 0.1:
    opt.zero_grad()
    outputs = model(**batch)
    next_word_logits = outputs.logits[:, num_virtual_tokens:-1, :]
    true_next_tokens = batch['input_ids'][:, 1:]
    loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
    print("Loss:", loss)
    loss.backward()
    opt.step()


#raise NotImplemented("Your task: iteratively train the model to reduce loss using prompt optimizer (opt)")


assert loss.item() <= 0.1
print("Good job!")

Loss: tensor(7.9844, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(6.8515, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(5.4433, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(4.4269, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(4.0675, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.6694, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.4031, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.2203, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.0787, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.9693, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.8791, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.7971, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.7181, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.6421, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.5693, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tens

In [33]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

for i in range(15):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))


Output: <s>A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it


### Parameter-efficient finetuning with LoRA (2 points)

When training on more serious tasks, you can use low-rank adapters based on the [LoRA paper](https://arxiv.org/pdf/2106.09685.pdf).

The core idea is to add low-rank adapters __in parallel with existing linear layers,__ like this:
<center><img src="https://i.imgur.com/6bQLNiG.png" width=240px></center>

In the original LoRA paper, the adapters were only added to attention projection matrices. However, [subsequent works](https://arxiv.org/abs/2305.14314) show that it is useful to adapt FFNs as well. But before we do any training, we need to implement the basic LoRA layer.

In [147]:
# re-load the model to remove any previous PEFT tuners
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 33/33 [00:13<00:00,  2.39it/s]
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [4]:
class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter. Wraps an existing OPT linear layer"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module  # pre-trained (frozen) linear layer
        self.adapter_A = nn.Parameter(torch.empty(module.in_features, rank, device=module.weight.device))
        nn.init.kaiming_uniform_(self.adapter_A, a=5 ** 0.5)
        self.adapter_B = nn.Parameter(torch.zeros(rank, module.out_features, device=module.weight.device))
        
    def forward(self, input):
        
        #print(self.module.in_features, self.module.out_features)
        # Apply self.module and LoRA adapter, return the sum (self.module outputs + adapter outputs)
        #  <YOUR CODE HERE>
        
        return self.module(input) + input.matmul(self.adapter_A).matmul(self.adapter_B)

In [149]:
# test your implementation
test_linear = nn.Linear(128, 128)
test_linear.weight.data[...] = torch.eye(128)
test_adapter = LoRALayer(test_linear, rank=8)

assert torch.allclose(test_adapter(torch.ones(1, 1, 128)), test_linear.bias + 1), "please check your forward pass"

test_adapter.adapter_A.data[...] = torch.linspace(0.1, -0.5, 128 * 8).view(128, 8)
test_adapter.adapter_B.data[...] = torch.linspace(0.5, -0.1, 128 * 8).view(8, 128)
test_linear.bias.data[...] = torch.linspace(1., -1., 128)

dummy_loss = F.mse_loss(test_adapter(torch.ones(1, 128) / 128).squeeze(), torch.linspace(-1, 1, 128))
assert torch.allclose(dummy_loss, torch.tensor(1.3711389), rtol=0, atol=1e-4)
dummy_loss.backward()
assert all(w.grad is not None for w in [test_adapter.adapter_A, test_adapter.adapter_B]), "some adapter weights have no grad"
assert torch.allclose(test_adapter.adapter_A.grad.sum(), torch.tensor(-0.60158), rtol=0, atol=1e-4), "bad grad w.r.t. A"
assert torch.allclose(test_adapter.adapter_B.grad.sum(), torch.tensor(0.9931), rtol=0, atol=1e-4), "bad grad w.r.t. B"
# note: bad grad means that your code is different from LoRA paper OR that your code is not autograd-friendly (e.g. no_grad)
del dummy_loss, test_linear, test_adapter
print("All tests passed!")

All tests passed!


### Apply LoRA to the model

The code below applies LoRA adapters on top of Q/K/V linear layers in Llama attention. You may also choose to modify other layers:
* self_attn.o_proj - attention output projection
* mlp.up_proj, mlp.gate_proj, mlp.down_proj - transformer feedforward layers
* lm_head - output LM head

__Note:__ please scroll down for the homework task

In [150]:
lora_rank = 8

for name, module in model.model.layers.named_modules():
    if 'LlamaDecoderLayer' in repr(type(module)):
        module.self_attn.q_proj = LoRALayer(module.self_attn.q_proj, rank=lora_rank).to(device)
        module.self_attn.k_proj = LoRALayer(module.self_attn.k_proj, rank=lora_rank).to(device)
        module.self_attn.v_proj = LoRALayer(module.self_attn.v_proj, rank=lora_rank).to(device)

assert sum(isinstance(module, LoRALayer) for module in model.modules()) == 96  # for Llama-7B

In [151]:
batch = tokenizer("This model wants to share its greatest secret:", return_tensors='pt', return_token_type_ids=False)
# test a single training step, make sure we get meaningful gradients
with torch.cuda.amp.autocast(dtype=torch.float32):
    out = model.forward(**batch)
    (out.logits.norm() / 100).backward()

for i, module in enumerate(model.modules()):
    if isinstance(module, LoRALayer):
        assert module.adapter_B.grad is not None
        assert module.adapter_B.grad.norm().item() > 0

model.zero_grad(set_to_none=True)
print("Grad check successful, well done!")

Grad check successful, well done!


### (example) How to train your model

The example below shows how to train the LoRA adapters on a dummy dataset. You will need to run a _similar_ training task later.

__Note:__ please scroll down for the homework task

In [152]:
# checking if the model can learn. Change max_steps for proper training
import datasets
data = datasets.load_dataset("Abirate/english_quotes", split="train[:32]") # 32 lines
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

trainer = transformers.Trainer(
    model=model, train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2, gradient_accumulation_steps=1,
        # note: if you want larger batch size, increase gradient_accumulation_steps
        warmup_steps=250, max_steps=100, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs', report_to=None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
# if you see cache warnings, set `model.config.use_cache = False` to silence them. Please re-enable for inference!

trainer.train()

# NOTE: this is just an example! you do not have to wait for this progressbar to finish :)

Map: 100%|██████████| 32/32 [00:00<00:00, 3354.10 examples/s]
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,1.225200
2,0.373100
3,1.409800
4,1.372100
5,0.838500
6,1.557300
7,1.770000
8,1.206600
9,0.546300
10,1.219200


You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.
/home/daozerova/.conda/envs/nlp/lib/python3.10/site-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


UnboundLocalError: local variable 'active_adapters' referenced before assignment

### Final task: *actually* train the model (4 points)

Your task is to fine-tune the model to _generate python code_. Please use the above examples for inspiration. More specifically,

* __dataset:__ use [codeparrot-clean](https://huggingface.co/datasets/codeparrot/codeparrot-clean) or any other data containing python code. Since you do not need much data for this excercise, it is enough to use just shorter validation subset of `codeparrots`
* __preprocessing:__ select python code based on file extentions (.py)  (may skip in case of codeparrot - it is 100% python)
* __short lines:__ please take the first 512 characters of each line
* __adapter type:__ please use LoRA as defined above __plus at least one of:__
   - extra adapter on lm_head
   - extra adapter on MLP components (mlp.*)
   - trainable input embeddings (requires tweaking memory usage)

* __training:__ you do not have to train to convergence. If all goes well, your model should `.generate` code after 500 steps. Please use batch size of at least 4 (4 x 1 x 512 tokens) using `gradient_accumulation_steps=4`.


Note: the peft library also has LoRA implementation. However, we ask that for this assignment you show at least one complete training run with your own LoRA code.

__Alternative assignment:__ Instead of doing python code, feel free to substitute the task with any other dataset, e.g. your favorite artist or podcast, as long as it's ethical. If you choose your own task, please show examples of what your model learned - or did not learn, akin to the code examples below.

In [5]:
from datasets import load_dataset

ds = load_dataset("codeparrot/codeparrot-clean-valid", split='train[:5000]')
ds[0]

Repo card metadata block was not found. Setting CardData to empty.


{'repo_name': 'pansapiens/mytardis',
 'path': 'tardis/apps/mx_views/views.py',
 'copies': '3',
 'size': '2892',
 'content': 'from django.conf import settings\nfrom django.core.paginator import Paginator, InvalidPage, EmptyPage\nfrom django.http import HttpResponse\n\nfrom tardis.tardis_portal.auth import decorators as authz\nfrom tardis.tardis_portal.models import Dataset\nfrom tardis.tardis_portal.shortcuts import get_experiment_referer\nfrom tardis.tardis_portal.shortcuts import render_response_index\n\n\n@authz.dataset_access_required\ndef view_full_dataset(request, dataset_id):\n    """Displays a MX Dataset and associated information.\n\n    Shows a full (hundreds of images) dataset its metadata and a list\n    of associated files with the option to show metadata of each file\n    and ways to download those files.  With write permission this page\n    also allows uploading and metadata editing.\n\n    Settings for this view:\n    INSTALLED_APPS += ("tardis.apps.mx_views",)\n    DAT

In [6]:
def truncate_characters(line):
    line['content'] = line['content'][:512]
    return line

ds = ds.map(truncate_characters)
ds[0], len(ds[0])

Map: 100%|██████████| 5000/5000 [00:00<00:00, 8659.09 examples/s] 


({'repo_name': 'pansapiens/mytardis',
  'path': 'tardis/apps/mx_views/views.py',
  'copies': '3',
  'size': '2892',
  'content': 'from django.conf import settings\nfrom django.core.paginator import Paginator, InvalidPage, EmptyPage\nfrom django.http import HttpResponse\n\nfrom tardis.tardis_portal.auth import decorators as authz\nfrom tardis.tardis_portal.models import Dataset\nfrom tardis.tardis_portal.shortcuts import get_experiment_referer\nfrom tardis.tardis_portal.shortcuts import render_response_index\n\n\n@authz.dataset_access_required\ndef view_full_dataset(request, dataset_id):\n    """Displays a MX Dataset and associated information.\n\n ',
  'license': 'bsd-3-clause',
  'hash': -8726488663588781404,
  'line_mean': 37.0526315789,
  'line_max': 79,
  'alpha_frac': 0.6549100968,
  'autogenerated': False},
 11)

In [34]:
lora_rank = 8
for name, module in model.model.layers.named_modules():
    if 'LlamaDecoderLayer' in repr(type(module)):
        module.self_attn.q_proj = LoRALayer(module.self_attn.q_proj, rank=lora_rank).to(device)
        module.self_attn.k_proj = LoRALayer(module.self_attn.k_proj, rank=lora_rank).to(device)
        module.self_attn.v_proj = LoRALayer(module.self_attn.v_proj, rank=lora_rank).to(device)
        #continue
    
    if 'LlamaMLP' in repr(type(module)):
        #print(name, module)
        module.gate_proj = LoRALayer(module.gate_proj, rank=lora_rank).to(device)
        module.up_proj = LoRALayer(module.up_proj, rank=lora_rank).to(device)
        module.down_proj = LoRALayer(module.down_proj, rank=lora_rank).to(device)

In [9]:
data = ds.map(lambda samples: tokenizer(samples['content']), batched=True)

Map: 100%|██████████| 5000/5000 [00:02<00:00, 2363.90 examples/s]


In [10]:
model._hf_peft_config_loaded = True 

trainer = transformers.Trainer(
    model=model, train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, gradient_accumulation_steps=4,
        # note: if you want larger batch size, increase gradient_accumulation_steps
        warmup_steps=250, max_steps=500, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs', report_to=None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/daozerova/.conda/envs/nlp/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Step,Training Loss
1,1.426600
2,1.556900
3,1.217400
4,1.561900
5,1.441800
6,1.468400
7,1.059900
8,1.302200
9,1.355000
10,1.404700


You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.
/home/daozerova/.conda/envs/nlp/lib/python3.10/site-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


UnboundLocalError: local variable 'active_adapters' referenced before assignment

### Generated by finetuned model: ###

In [15]:
prompts =  ['', 'import', 'from', 'while', 'try', 'if', 'for', 'torch']  # feel free to add a few more that are not 100% assiciated with Python
after_finetuning = []

for prompt in prompts:
    print('Given prompt', prompt)
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

    for i in range(100):
        next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

    after_finetuning.append(tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))
    print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))
    
# <A WHOLE LOT OF YOUR CODE>

Given prompt 

Output: <s># -*- coding: utf-8 -*-
#
# Copyright 2014-2015 The Chromium Authors. All rights reserved.
# Use of this source code is governed by a BSD-style license that can be
# found in the LICENSE file.

"""
Tests for the 'chrome' command.
"""

from __future__ import print_function

import
Given prompt import

Output: <s>import os
import sys
import time
import json
import logging
import threading
import Queue
import thread
import traceback
import socket
import struct
import re
import urllib
import urllib2
import urlparse
import httplib
import httplib2
import ssl
import base64
import random
import datetime
import hashlib
import hmac
import binascii
import email.utils
import email.message
import email.
Given prompt from

Output: <s>from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import sys
import time
import json
import logging
import threading
import traceback


### Generated by not finetuned model: ###

In [24]:
before_finetuning = []
for prompt in prompts:
    print('Given prompt', prompt)
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

    with torch.no_grad():
        generated_ans = model.generate(**batch, max_length=100)
        #print(generated_ans, tokenizer.decode(generated_ans.cpu().numpy()[0]))
        print("\nOutput:", tokenizer.decode(generated_ans.cpu().numpy()[0]))
        before_finetuning.append(tokenizer.decode(generated_ans.cpu().numpy()[0]))

Given prompt 

Output: <s> ▶▶ 2019-2020 School Year
The 2019-2020 school year is here! We are so excited to welcome our new students and families to the school. We are also excited to welcome back our returning families.
We are looking forward to another great year at the school. We are also looking forward to another great year of learning and growing together.
We are also looking forward to another great year of learning and growing
Given prompt import

Output: <s> import Foundation

public extension NSURL {
    public var absoluteString: String {
        return String(cString: CFBundleGetBundleWithURL(self).UTF8String)
    }
}</s>
Given prompt from

Output: <s>from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys

from absl import flags

from tensorflow.python import pywrap_tensorflow
from tensorflow.python.eager import context
from tensorflow.python.eager import function
from tensorflow.python.eager import

In [30]:
# This template helps to compare generated code samples in pretty table form
# feel free to present your work in other forms

from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''



In [33]:
rows = []

for i in range(len(prompts)):
    # replace placeholders in the format() arguments
    rows.append(row_template.format(prompts[i], before_finetuning[i][3:], after_finetuning[i][3:]))

display(HTML(table_template.format('\n'.join(rows))))

PROMPT,BEFORE,AFTER
``,▶▶ 2019-2020 School Year The 2019-2020 school year is here! We are so excited to welcome our new students and families to the school. We are also excited to welcome back our returning families. We are looking forward to another great year at the school. We are also looking forward to another great year of learning and growing together. We are also looking forward to another great year of learning and growing,"# -*- coding: utf-8 -*- # # Copyright 2014-2015 The Chromium Authors. All rights reserved. # Use of this source code is governed by a BSD-style license that can be # found in the LICENSE file. """""" Tests for the 'chrome' command. """""" from __future__ import print_function import"
`import`,import Foundation public extension NSURL { public var absoluteString: String { return String(cString: CFBundleGetBundleWithURL(self).UTF8String) } },import os import sys import time import json import logging import threading import Queue import thread import traceback import socket import struct import re import urllib import urllib2 import urlparse import httplib import httplib2 import ssl import base64 import random import datetime import hashlib import hmac import binascii import email.utils import email.message import email.
`from`,from __future__ import absolute_import from __future__ import division from __future__ import print_function import os import sys from absl import flags from tensorflow.python import pywrap_tensorflow from tensorflow.python.eager import context from tensorflow.python.eager import function from tensorflow.python.eager import test from tensorflow.python.eager import backprop from tensorflow.python,from __future__ import absolute_import from __future__ import division from __future__ import print_function from __future__ import unicode_literals import os import sys import time import json import logging import threading import traceback import re import datetime import functools import inspect import signal import signal import traceback import logging.handlers import logging.config import logging.
`while`,"while(1) while(1) { // do something } \end{code} Comment: This is not the same as the OP's code. Comment: @Jeffrey: It's the same as the OP's code, except that it's not a function. Comment: @Jeffrey: The OP's code is a function, but it's not a function declaration. Comment","while True: try: line = input(""> "") if line == ""exit"": break if line == ""help"": print(""Usage: python3 -m python3_game"") break if line == ""start"": print(""Starting game..."") game_loop() elif line == ""stop"": print(""Stopping game..."") game_loop() elif"
`try`,try to find the best solution for your needs. We are a team of professionals with a long experience in the field of web development. We are a team of professionals with a long experience in the field of web development. We are a team of professionals with a long experience in the field of web development. We are a team of professionals with a long experience in the field of web development. We are a team of professionals with a long experience in the field of,try: from urllib.parse import urlparse except ImportError: from urlparse import urlparse from django.conf import settings from django.core.exceptions import ImproperlyConfigured from django.http import HttpResponse from django.utils.translation import ugettext_lazy as _ from oscar.core.loading import get_model from oscar.core.utils import get_file_size
`if`,"if ( !window.atmosphere ) { window.atmosphere = {}; } (function () { var o = atmosphere.util, atmosphere = atmosphere.atmosphere = function () { var _isClosed = false, _isOpening = false, _isOpen = false, _isClosing = false, _isError = false, _is",if ( ! defined( 'ABSPATH' ) ) exit; /** * Class WPML_Translation_Management_Controller * * @since 1.2.0 * @author Piotr Koczanowski */ class WPML_Translation_Management_Controller { /** * @var WPML_Translation_Management_Model */
`for`,for the 2019-2020 school year. The application process 

If you reach this: congratulations! you've completed everything in this practice session.

If you want to dig deeper, try to implement prompt-tuning (for bonus points!).
You can read more about prompt tuning variants in paper [1](https://arxiv.org/abs/2104.08691) or paper [2](https://arxiv.org/abs/2101.00190). Both versions can be implemented by passing trainable prompts as `model.forward(..., past_key_values=your_prompts)`.



### Read more

* How post-training quantization works: https://arxiv.org/abs/2208.07339
* An overview of running large models: https://huggingface.co/docs/accelerate/package_reference/big_modeling
* A general library for different adapter types: https://adapterhub.ml/


### [extra info] Running other models.

This notebook's code can run with other models of similar size, such as [Falcon-7B](https://huggingface.co/tiiuae/falcon-7b), [OPT-6.7B](https://huggingface.co/facebook/opt-6.7b) or [BLOOM-7.1B](https://huggingface.co/bigscience/bloom-7b1). However, they will require minor code tweaks:
1. change the model name in `AutoModelForCausalLM.from_pretrained()` __and__ `AutoTokenizer`
2. In the prompt tuning code, change `model.model.embed_tokens` to refer to the target model's word embeddings. Simply `print(model)` to navigate to them.
3. Change code to add Lora layers - specifically where you what the transformer block components, since those components now have different names.